#### 1. Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный).

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from pymongo import MongoClient, errors
from pprint import pprint
import json
import re
import time

In [2]:
with open('email.json', 'r') as f:
    email_data = json.load(f)

In [4]:
driver = webdriver.Chrome()  
driver.get('https://mail.ru/')
assert 'Mail.ru: почта' in driver.title

driver.find_element_by_id('mailbox:login').send_keys(email_data['login'])
Select(driver.find_element_by_id('mailbox:domain')).select_by_visible_text('@mail.ru')
driver.find_element_by_id('mailbox:submit').click()
time.sleep(3)

driver.find_element_by_id('mailbox:password').send_keys(email_data['password'])
driver.find_element_by_id('mailbox:submit').click()
time.sleep(3)

WebDriverWait(driver, 5).until(EC.title_contains('Входящие'))
assert 'Входящие' in driver.title

mails = []

for i in range(1):
    time.sleep(3)
    items = driver.find_elements_by_xpath("//a[contains(@class,'llc js-tooltip-direction_letter-bottom js-letter-list-item llc_pony-mode llc_normal')]")
    actions = ActionChains(driver)
    
    for item in items:
        link = item.get_attribute('href')
        info = item.find_element_by_class_name('llc__content')
        author = info.find_element_by_class_name('ll-crpt').get_attribute('title')
        subject = info.find_element_by_class_name('ll-sj__normal').text
        date = item.find_element_by_class_name('llc__item_date').get_attribute('title')
        mail = {'author': author, 'date': date, 'subject': subject, 'link': link}
        mails.append(mail)    
    
    actions.move_to_element(items[-1])
    actions.perform()
    

for mail in mails:
    driver.get(mail['link'])
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[contains(@id, '_BODY')]")))
    mail['text'] = driver.find_element_by_xpath("//*[contains(@id, '_BODY')]").text
    driver.back()
    WebDriverWait(driver, 5).until(EC.title_contains('Входящие'))

driver.quit()

In [5]:
db_name = 'mails_db'
client = MongoClient('mongodb://127.0.0.1:27017')
db = client[db_name]
db.mails_db.insert_many(mails)

#### 2. Написать программу, которая собирает «Хиты продаж» с сайта техники mvideo и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары.

In [6]:
chrome_options = Options()
chrome_options.add_argument('start-maximized')

driver = webdriver.Chrome(options = chrome_options)
driver.get('https://www.mvideo.ru/')
assert 'М.Видео' in driver.title

ajax_category_carousel = driver.find_elements_by_xpath("//div[contains(@data-init, 'ajax-category-carousel')]")

hits = None
for element in ajax_category_carousel:
    if 'Хиты продаж' in element.text:
        hits = element
        break

goods = []
for _ in range(5):
    time.sleep(5)
    items = hits.find_elements_by_class_name('gallery-list-item')
    for item in items:
        good = {}
        name = item.find_element_by_class_name('sel-product-tile-title').text
        link = item.find_element_by_class_name('sel-product-tile-title').get_attribute('href')
        price = re.sub('\D', '', item.find_element_by_class_name('c-pdp-price__current').text)
        good['name'] = name
        good['link'] = link
        good['price'] = price
        if name != '' and good not in goods:
            goods.append(good)
    button = WebDriverWait(hits, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'sel-hits-button-next')))
    button.send_keys(Keys.ENTER)

driver.quit()


In [7]:
goods

[{'name': 'Смартфон Apple iPhone 11 64GB Purple (MWLX2RU/A)',
  'link': 'https://www.mvideo.ru/products/smartfon-apple-iphone-11-64gb-purple-mwlx2ru-a-30045421',
  'price': '59990'},
 {'name': 'Наушники Apple AirPods Pro with Wireless Case (MWP22RU/A)',
  'link': 'https://www.mvideo.ru/products/naushniki-apple-airpods-pro-with-wireless-case-mwp22ru-a-50131384',
  'price': '18990'},
 {'name': 'Планшет Apple iPad 10.2" 32Gb Wi-Fi Space Grey (MW742RU/A)',
  'link': 'https://www.mvideo.ru/products/planshet-apple-ipad-10-2-32gb-wi-fi-space-grey-mw742ru-a-30045494',
  'price': '27990'},
 {'name': 'Наушники Apple AirPods w/Charging Case (MV7N2RU/A)',
  'link': 'https://www.mvideo.ru/products/naushniki-apple-airpods-w-charging-case-mv7n2ru-a-50126638',
  'price': '11490'},
 {'name': 'Наушники Apple AirPods w/Wireless Charg.Case MRXJ2',
  'link': 'https://www.mvideo.ru/products/naushniki-apple-airpods-w-wireless-charg-case-mrxj2-50126637',
  'price': '13990'},
 {'name': 'Смартфон Samsung Galaxy

In [8]:
db_name = 'goods_db'
client = MongoClient('mongodb://127.0.0.1:27017')
db = client[db_name]
db.goods_db.insert_many(goods)